In [8]:
import numpy as np
from Load_data import load_data
from Utils import encode_results
import os
print(os.getcwd())

ModuleNotFoundError: No module named 'Utils'

In [1]:
data = load_data('20242025_premier_league.csv')
data['result'] = encode_results(data['FTR'])

NameError: name 'load_data' is not defined